# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
`
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'.
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
#Carregar base 'sinasc'
sinasc = pd.read_csv('SINASC_RO_2019.csv')
#Número de registros
registros,_ = sinasc.shape
print("O número de registros é %2d." %registros)

O número de registros é 13788.


<ipython-input-2-9afdfd7e3243>:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  sinasc = pd.read_csv('SINASC_RO_2019.csv')


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Duplicados
dups = sinasc.duplicated()
print("O número de linhas duplicadas é %2d." %sum(dups))

O número de linhas duplicadas é  0.


In [5]:
sinasc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13788 entries, 0 to 13787
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ORIGEM        13788 non-null  int64  
 1   CODESTAB      13732 non-null  float64
 2   CODMUNNASC    13788 non-null  int64  
 3   LOCNASC       13788 non-null  int64  
 4   IDADEMAE      13788 non-null  int64  
 5   ESTCIVMAE     13637 non-null  float64
 6   ESCMAE        13631 non-null  object 
 7   CODOCUPMAE    12339 non-null  object 
 8   QTDFILVIVO    12886 non-null  float64
 9   QTDFILMORT    12587 non-null  float64
 10  CODMUNRES     13787 non-null  float64
 11  GESTACAO      13145 non-null  object 
 12  GRAVIDEZ      13750 non-null  object 
 13  PARTO         13764 non-null  object 
 14  CONSULTAS     13787 non-null  float64
 15  DTNASC        13787 non-null  object 
 16  HORANASC      13778 non-null  float64
 17  SEXO          13784 non-null  object 
 18  APGAR1        13743 non-nu

In [6]:
# 2) seu código aqui
missing = sinasc.isnull().sum()
missing

,0
ORIGEM,0
CODESTAB,56
CODMUNNASC,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,151
ESCMAE,157
CODOCUPMAE,1449
QTDFILVIVO,902
QTDFILMORT,1201


In [7]:
# 3) seu código aqui
#Colunas interesse: ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO',   'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
sinasc = sinasc[['LOCNASC',
                 'IDADEMAE',
                 'ESTCIVMAE',
                 'ESCMAE',
                 'QTDFILVIVO',
                 'GESTACAO',
                 'GRAVIDEZ',
                 'CONSULTAS',
                 'APGAR5']]
sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
13783,1,29,2.0,8 a 11 anos,1.0,42 semanas e mais,Única,3.0,10.0
13784,1,39,2.0,1 a 3 anos,3.0,37 a 41 semanas,Única,4.0,10.0
13785,1,29,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4.0,10.0
13786,1,17,2.0,4 a 7 anos,0.0,37 a 41 semanas,Única,4.0,10.0


In [8]:
# 4) seu código aqui
#Remover missings APGAR5
APGAR5_missing = sinasc['APGAR5'].isnull().sum()
print("A variável APGAR5 contém %d entratadas faltantes." %APGAR5_missing)
#Contar o número de linhas e de missings

A variável APGAR5 contém 51 entratadas faltantes.


In [ ]:
sinasc = sinasc.dropna(subset = ['APGAR5'])
APGAR5_missing = sinasc['APGAR5'].isnull().sum()
print("A variável APGAR5 contém %d entratadas faltantes." %APGAR5_missing)

In [9]:
# 5) seu código aqui
#['ESTCIVMAE', 'CONSULTAS'] código 9
sinasc.loc[:,'ESTCIVMAE'] = sinasc['ESTCIVMAE'].fillna(9)
sinasc.loc[:,'CONSULTAS'] = sinasc['CONSULTAS'].fillna(9)
missing_E = sinasc['ESTCIVMAE'].isnull().sum()
missing_C = sinasc['CONSULTAS'].isnull().sum()
print("A variável ESTCIVMAE contém %d entratadas faltantes." %missing_E)
print("A variável CONSULTAS contém %d entratadas faltantes." %missing_C)

A variável ESTCIVMAE contém 0 entratadas faltantes.
A variável CONSULTAS contém 0 entratadas faltantes.


In [10]:
# 6) Seu código aqui
#QTDFILVIVO -> substituir dados faltantes por 0
sinasc.loc[:,'QTDFILVIVO'] = sinasc['QTDFILVIVO'].fillna(0)
missing_Q = sinasc['QTDFILVIVO'].isnull().sum()
print("A variável QTDFILVIVO contém %d entratadas faltantes." %missing_Q)

A variável QTDFILVIVO contém 0 entratadas faltantes.


In [11]:
# 7) seu código aqui
sinasc.isnull().sum()

,0
LOCNASC,0
IDADEMAE,0
ESTCIVMAE,0
ESCMAE,157
QTDFILVIVO,0
GESTACAO,643
GRAVIDEZ,38
CONSULTAS,0
APGAR5,51


In [12]:
#Para manter a coerência do conjunto de dados, optei por utilizar os mesmos valores apresentados na documentação
#ESCMAE -> 9: ignorado
sinasc.loc[:,'ESCMAE'] = sinasc['ESCMAE'].fillna(9)
missing_ESC = sinasc['ESCMAE'].isnull().sum()
print("A variável ESCMAE contém %d entratadas faltantes." %missing_ESC)

#GESTACAO -> 9: ignorado
sinasc.loc[:,'GESTACAO'] = sinasc['GESTACAO'].fillna(9)
missing_GE = sinasc['GESTACAO'].isnull().sum()
print("A variável GESTACAO contém %d entratadas faltantes." %missing_GE)

#GRAVIDEZ -> 9: ignorado
sinasc.loc[:,'GRAVIDEZ'] = sinasc['GRAVIDEZ'].fillna(9)
missing_GR = sinasc['GRAVIDEZ'].isnull().sum()
print("A variável GRAVIDEZ contém %d entratadas faltantes." %missing_GR)

A variável ESCMAE contém 0 entratadas faltantes.
A variável GESTACAO contém 0 entratadas faltantes.
A variável GRAVIDEZ contém 0 entratadas faltantes.


In [13]:
# 8) seu código aqui
#APGAR1 e APGAR5

#Entre 8 e 10 está em uma faixa 'normal'.
#Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
#Entre 4 e 5 significa 'asfixia moderada'.
#Entre 0 e 3 significa 'asfixia severa'.
sinasc_copia = sinasc.copy()

sinasc_copia.loc[sinasc_copia['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'
sinasc_copia.loc[(sinasc_copia['APGAR5'] == 4) | (sinasc['APGAR5'] == 5), 'APGAR5_CAT'] = 'asfixia moderada'
sinasc_copia.loc[(sinasc_copia['APGAR5'] == 6) | (sinasc['APGAR5'] == 7), 'APGAR5_CAT'] = 'asfixia leve'
sinasc_copia.loc[sinasc['APGAR5'] >= 8, 'APGAR5_CAT'] = 'normal'

sinasc

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
13783,1,29,2.0,8 a 11 anos,1.0,42 semanas e mais,Única,3.0,10.0
13784,1,39,2.0,1 a 3 anos,3.0,37 a 41 semanas,Única,4.0,10.0
13785,1,29,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4.0,10.0
13786,1,17,2.0,4 a 7 anos,0.0,37 a 41 semanas,Única,4.0,10.0


In [14]:
# 9) seu código aqui
#inplace = True
sinasc.rename(columns = {"LOCNASC": "loc_nasc",
                        "IDADEMAE": "idade_mae",
                        "ESTCIVMAE": "est_civ_mae",
                        "ESCMAE": "esc_mae",
                        "QTDFILVIVO": "qtd_fil_vivo",
                        "GESTACAO": "gestacao",
                        "GRAVIDEZ": "gravidez",
                        "CONSULTAS": "consultas",
                        "APGAR5": "apgar5",
                        "APGAR5_CAT": "apgar5_cat"
                        }, inplace = True)
sinasc

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4.0,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4.0,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3.0,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4.0,10.0
...,...,...,...,...,...,...,...,...,...
13783,1,29,2.0,8 a 11 anos,1.0,42 semanas e mais,Única,3.0,10.0
13784,1,39,2.0,1 a 3 anos,3.0,37 a 41 semanas,Única,4.0,10.0
13785,1,29,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4.0,10.0
13786,1,17,2.0,4 a 7 anos,0.0,37 a 41 semanas,Única,4.0,10.0
